# Sub-section participation using a NSHM Composite Solution 

exactly what Weka/Kororaa do in current Rupture Explorer

In [247]:
#import pathlib
import json
import nzshm_model as nm
import geopandas as gpd
import solvis

from solvis.fault_system_solution_helper import FaultSystemSolutionHelper
from solvis_graphql_api.color_scale import ColourScaleNormaliseEnum, get_colour_values
from ipyleaflet import Map, GeoJSON

In [258]:
# load the composite_solution and shoose the Cruslal FaultSystemSolution
current_model = nm.get_model_version("NSHM_v1.0.4")
slt = current_model.source_logic_tree
csol = solvis.CompositeSolution.from_archive("NSHM_v1.0.4_CompositeSolution.zip", slt)
fss = csol._solutions['CRU']

helper = FaultSystemSolutionHelper(fss)

## Choose some faults and get their unique rupture IDs

In [273]:
# get ruptures passing through named parent fault(s)
TARGET_FAULTS = ['Wellington Hutt Valley: 1', 'Wellington Hutt Valley: 2', 'Wairarapa: 1'] #, 'Pokeno', 'BooBoo', 'Masterton'

ruptures = helper.ruptures_for_parent_fault_names(TARGET_FAULTS) # 
df0 = fss.rs_with_rupture_rates
rupture_sections_df = df0[df0["Rupture Index"].isin(ruptures)]

In [274]:
## post-helper approach
subsections = helper.subsections_for_ruptures(ruptures)
section_rates = fss.rs_with_rupture_rates[["Rupture Index", "section", "rate_weighted_mean"]]\
    .groupby("section").agg('sum')["rate_weighted_mean"]
mro_sections_rates = section_rates[section_rates.index.isin(subsections)]

In [275]:

# get some stats
section_count = len(rupture_sections_df['section'].unique())
print(f'the faults in {TARGET_FAULTS} have:')
print(f' {len(ruptures)} unique ruptures...')
print(f' {section_count} unique fault sections...')
print()

the faults in ['Wellington Hutt Valley: 1', 'Wellington Hutt Valley: 2', 'Wairarapa: 1'] have:
 301 unique ruptures...
 443 unique fault sections...



### A) Calculate sub-section participation rate (solvis)

In [276]:
#compare old-new
assert set(subsections) == set(rupture_sections_df['section'].unique())

section_rates = fss.rs_with_rupture_rates[["Rupture Index", "section", "rate_weighted_mean"]]\
    .groupby("section").agg('sum')["rate_weighted_mean"]
mro_sections_rates = section_rates[section_rates.index.isin(subsections)]

### B) Calculate sub-section participation rate (solvis-graphql-api)

In [277]:
# the following comes from SGI.cached.fault_section_aggregates_gdf
fsr = fss.fault_sections_with_rupture_rates
fsr = fsr[fsr.section.isin(subsections)]
section_aggregates = fsr.pivot_table(
        index=['section'],
        aggfunc=dict(rate_weighted_mean=['sum', 'min', 'max', 'mean'], Magnitude=['count', 'min', 'max', 'mean']),
    )

section_aggregates.columns = [".".join(a) for a in section_aggregates.columns.to_flat_index()]

# this is for illustration - showing the  resulting rates fir FSS are the same
section_aggregates['section_participation_rate'] = mro_sections_rates

### here'we can see that we have the participation rate two ways
sa_df = section_aggregates.drop(columns=['Magnitude.max', 'Magnitude.min', 'Magnitude.count', 'Magnitude.mean'])
sa_df[sa_df.index==1426]

,rate_weighted_mean.max,rate_weighted_mean.mean,rate_weighted_mean.min,rate_weighted_mean.sum,section_participation_rate
section,,,,,
1426.0,0.000041,0.000005,1.385813e-15,0.000172,0.000172


In [262]:
# join the fault sufaces (with geometry) and aggregated rates 
section_aggregates_detail = section_aggregates.join(fss.fault_surfaces(), 'section', how='inner', rsuffix='_R')
# section_aggregates_detail.head()

### Style the geojson using a color scale

In [263]:
# code from SGI.composite_solution.composite_rupture_sections.py 
fault_sections_gdf = gpd.GeoDataFrame(section_aggregates_detail)
color_values = get_colour_values(
                color_scale="inferno",
                color_scale_vmax=fault_sections_gdf['rate_weighted_mean.sum'].max(),
                color_scale_vmin=fault_sections_gdf['rate_weighted_mean.sum'].min(),
                color_scale_normalise= ColourScaleNormaliseEnum.LOG.value,
                values=tuple(fault_sections_gdf['rate_weighted_mean.sum'].tolist()),
            )
# print(color_values)
data = json.loads(fault_sections_gdf.to_json())

# merge the styling with the geojson
for feature, color in zip(data["features"], color_values):
    feature["properties"]["style"] = {
        "color": "black",
        "weight": 1,
        "fillColor": color,
        "fillOpacity": 1,
    }

### Display with ipyleaflet

In [264]:
center = [-41.5, 175]
zoom = 7
map = Map(center=center, zoom=zoom)
g = GeoJSON(data=data)
map.add(g)
map

Map(center=[-41.5, 175], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [257]:
# section_aggregates_detail[['section_participation_rate']]